# Scraping and Parsing: EAD XML Finding Aids from the Library of Congress

In [ ]:
import os
from urllib.request import urlopen
from bs4 import BeautifulSoup
import subprocess

In [ ]:
## Creating a directory called 'LOC_Metadata' and setting it as our current working directory

!mkdir /sharedfolder/LOC_Metadata

os.chdir('/sharedfolder/LOC_Metadata')

In [ ]:
## To make this notebook self-contained, we'll download a list of XML finding aid files the 'right' way.
##  (In practice I normally use the 'find-and-replace + grep + wget' approach we covered in class,
##   because it takes some extra effort to remind myself how to parse the HTML page via BeautifulSoup.)

## We first load a page with links to finding aids in the 'recorded sound' collection.

finding_aid_list_url = 'http://findingaids.loc.gov/source/RS'

finding_aid_list_page = urlopen(finding_aid_list_url).read().decode('utf8')  # Loading the page

print(finding_aid_list_page[:700])  # Printing the first 700 characters in the page we just loaded

In [ ]:
## Now we'll parse the page's HTML using BeautifulSoup ...

soup = BeautifulSoup(finding_aid_list_page, 'lxml')

## ... and examine soup.find_all('a'), which returns a list of 'a' elements (i.e., HTML links).

print(len(soup.find_all('a')))         # Checking the number of links on the page

print()                                # Printing a blank line for readability

print(soup.find_all('a')[70])          # Printing element #70 in the list

In [ ]:
## We can access the 'href' attribute of an element (i.e., the link URL) using 'href' in 
## brackets, just like a dictionary.

soup.find_all('a')[70]['href']

In [ ]:
## Now let's make a list of every link on the page.

all_links = []

for element in soup.find_all('a'):         # Looping through all 'a' elements.
    try:                                   # Because some 'a' elements do not contain 'href' attributes, 
        all_links.append(element['href'])  ## we can use a try/except statement to skip elements that 
    except:                                ## would otherwise raise an error.
        pass

all_links[:15]                             # Outputting the first 15 links in the list

In [ ]:
## We know that the URL for every XML file we're looking for ends in '.2', so we can
## use that fact to filter out irrelevant links.

xml_urls = []

for link in all_links:
    if link[-2:] == '.2':           # Checking whether the last two characters of a link are '.2'
        xml_urls.append(link)

xml_urls                            # Outputting the full list of relevant XML URLs    

In [ ]:
## Downloading each XML file in our list of URLs

## We can use the subprocess module (which we imported above) to issue commands in the bash shell.
## In an interactive bash shell session we'd use spaces to separate arguments; instead, subprocess
## takes arguments in the form of a Python list.

## For each item in our list, the following issues a command with two arguments: 'wget' followed by the URL.
## It thus downloads each XML file to the current directory.

for url in xml_urls:
    subprocess.call(['wget', url])

In [ ]:
## Outputting a list of filenames in the current directory

## In Unix-like operating systems, './' always refers to the current directory.

os.listdir('./')

In [ ]:
## Just in case there are other files in the current directory, we can use a 
## list comprehension to create a list of filenames that end in '.2' and assign
## it to the variable 'xml_filenames'.

xml_filenames = [item for item in os.listdir('./') if item[-2:]=='.2']

xml_filenames

In [ ]:
## Now let's choose an arbitrary XML file in our collection so we can figure out how to parse it.

xml_filename = xml_filenames[4]       ## Selecting filename #4 in our list

xml_text = open(xml_filename).read()  ## Reading the file and assigning its content to the variable 'xml_text'

print(xml_text[:700])                 ## Printing the first 700 characters in the XML text we just loaded

In [ ]:
## Parse the XML text from the previous cell using Beautiful Soup

soup = BeautifulSoup(xml_text, 'lxml')

In [ ]:
## By looking at the XML text above, we can see that the 'ead' element is the root of our XML tree.
## Let's use a for loop to look at the names of elements one next level down in the tree.

for element in soup.ead:
    print(element.name)

In [ ]:
## In practice you'd usually just look through the XML file by eye, identify the elements 
## you're looking for, and use soup.find_all('...') to extract them. For now, let's continue 
## working down the XML tree with BeautifulSoup.

# You can find a glossary of EAD element names here:
# https://loc.gov/ead/EAD3taglib/index.html

In [ ]:
## Since the 'eadheader' element is administrative metadata we don't care about, let's 
## repeat the process for 'soup.ead.archdesc' ('archdesc' is 'archival description' in EAD parlance).

for element in soup.ead.archdesc:
    if element.name != None:  ## Filtering out 'None' elements, which in this case are irrelevant comments
        print(element.name)

In [ ]:
## By looking at the XML file in a text editor, I notice the 'did' element ('descriptive identification')
## contains the item-level information we're looking for. Let's run another for loop to look at the 
## names of elements contained within each 'did' element.

for element in soup.ead.archdesc.did:
    if element.name != None:
        print(element.name)

## Note that 'soup.ead.archdesc.did' only refers to the first 'did' element in the XML document.

In [ ]:
## OK, that's enough exploring. Let's use soup.find_all() to create a list of 'did' elements. 

did_elements = soup.find_all('did')

print(len(did_elements))           ## Printing the number of 'did' elements in our list

print()

print(did_elements[4])             ## Printing item #4 in the the list

In [ ]:
## Not every 'did' element contains the same fields; different objects are described differently.

## Try running this cell several times, plugging in other index numbers to compare the way
## different items' records are formatted.

print(did_elements[7])

In [ ]:
## If you run the cell above several times with different index numbers, you'll notice that the 
## first item in the list (index 0) refers to the entire box of records, while the others are 
## individual folders or series of folders.

## To make things more complicated, some items are physically described using 'container' elements 
## while others use 'extent' instead. Most appear to include 'unittitle' and 'unitdate'.

## Our goal is to create a CSV that contains a basic description of each 'unit', or 'did' element,
## in each XML finding aid. For the purposes of this exercise, let's include the following pieces 
## of information for each unit, where available:

#### title of the source collection
#### unittitle
#### unitdate
#### container type
#### container number
#### extent

In [ ]:
## Since each XML finding aid represents a single collection, we'll want to include a column that 
## identifies which collection it comes from. By reading through the XML files, we see that each 
## has a single element called 'titleproper' that describes the whole collection.

## Let's create a recipe to extract that text. Here's a first try:

collection_title = soup.find('titleproper').get_text()

collection_title

In [ ]:
## That format is OK, but we should remove the tab and newline characters. Let's try again, using 
## the replace() function to replace them with spaces.

collection_title = soup.find('titleproper').get_text().replace('\t', ' ').replace('\n', ' ')

collection_title

In [ ]:
## We can add the strip() function to remove the space at the end of the string.

collection_title = soup.find('titleproper').get_text().replace('\t', ' ').replace('\n', ' ').strip()

collection_title

In [ ]:
## We still have a series of spaces in a row in the middle of the string. We can use a 'while loop' 
## to repeatedly replace any occurrence of '  ' (two spaces) with ' ' (one space).

collection_title = soup.find('titleproper').get_text().replace('\t', ' ').replace('\n', ' ').strip()

while '  ' in collection_title:
    collection_title = collection_title.replace('  ', ' ')

collection_title

In [ ]:
## Perfect. We'll extract the collection name whenever we open an XML finding aid and include it 
## in each CSV row associated with that collection.

In [ ]:
## Now on to 'unittitle'. Recall that we created a list of 'did' elements above, called 'did_elements'.

element = did_elements[4]

unittitle = element.find('unittitle').get_text()

unittitle

In [ ]:
## Since those tabs and newlines are a recurring probem, we should define a function that 
## removes them from any given text string.

def clean_text(text):
    temp_text = text.replace('\t', ' ').replace('\n', ' ').strip()
    while '  ' in temp_text:
        temp_text = temp_text.replace('  ', ' ')
    return temp_text

In [ ]:
# Let's test our clean_text() function.

element = did_elements[4]

unittitle = element.find('unittitle').get_text()

unittitle = clean_text(unittitle)

unittitle

In [ ]:
## Now let's try extracting the 'unittitle' field for each 'did' element in our list.

for element in did_elements:
    unittitle = element.get_text().replace('\t', ' ').replace('\n', ' ').strip()
    print(clean_text(unittitle))
    print('-----------------')       # Printing a divider between elements

In [ ]:
## The first element in the list above contains more information than we need, but we can
## let that slide for this exercise.

## Next is 'unitdate'. We'll use our clean_text() function once again.

element = did_elements[4]

unitdate = element.find('unitdate').get_text()

unitdate = clean_text(unitdate)

unitdate

In [ ]:
## Let's loop through the list of 'did' elements and see if our 'unittitle' recipe holds up.

for element in did_elements:
    unitdate = element.find('unitdate').get_text()
    print(clean_text(unitdate))
    print('-----------------')       # Printing a divider between elements

In [ ]:
## Now on to container type and number. Let's examine a 'container' XML element.

element = did_elements[4]

element.find('container')

In [ ]:
## Since the container type ('folder', in this case) is an attribute in the 'container' tag, 
## we can extract it using bracket notation.

element = did_elements[4]

container_type = element.find('container')['type']

container_type

In [ ]:
## The container number is specified between the opening and closing 'container' tags, 
## so we can get it using get_text().

element = did_elements[4]

container_number = element.find('container').get_text()

container_number

In [ ]:
## Next we'll try to get the container type and number for each 'did' element in our list ...

for element in did_elements:
    container_type = element.find('container')['type']
    print(container_type)

    container_number = element.find('container').get_text()
    print(container_number)

    print('-----------------')       # Printing a divider between elements

## ... and we get an error. The reason is that some 'did' elements don't include a 'container' field.

In [ ]:
## Using try/accept notation, whenever we get an error because a container element isn't found,
## we can revert to '' (an empty string) instead.

for element in did_elements:
    try:
        container_type = element.find('container')['type']
    except:
        container_type = ''
    print(container_type)
    
    try:
        container_number = element.find('container').get_text()
    except:
        container_number = ''
    print(container_number)
    print('-----------------')       # Printing a divider between elements

In [ ]:
## The last field we'll extract is 'extent', which is only included in a handful of 'did' elements.

element = did_elements[3]

extent = element.find('extent').get_text()

extent

In [ ]:
## Let's extract 'extent' from each element in our list of 'did' elements (for those that happen to include it).

for element in did_elements:
    try:
        extent = element.find('extent').get_text()
    except:
        extent = ''
    print(extent)
    print('-----------------')       # Printing a divider between elements

In [ ]:
## Let's put it all together and view our chosen fields for a single 'did' element.
## We will combine our fields in a list to create a 'row' for our future CSV file.

element = did_elements[6]

# unittitle
try:                           # Added try/except statements for 'unittitle' and 'unitdate' just to be safe
    unittitle = clean_text(element.find('unittitle').get_text())
except:
    unittitle = ''
    
# unitdate
try:
    unitdate = clean_text(element.find('unitdate').get_text())
except:
    unitdate = ''
    
# container type and number
try:
    container_type = element.find('container')['type']
except:
    container_type = ''

try:
    container_number = element.find('container').get_text()
except:
    container_number = ''

# extent
try:
    extent = element.find('extent').get_text()
except:
    extent = ''

row = [unittitle, unitdate, container_type, container_number, extent]


print(row)

In [ ]:
## Let's take a step back and generalize, so that we can extract metadata for each 
## 'did' element in a single XML file.

## We will also include the 'collection title' field ('titleproper' in EAD's vocabulary) as 
## the first item in each row.

xml_filename = xml_filenames[3] # <-- Change the index number there to run the script on another XML file in the list.


xml_text = open(xml_filename).read()

soup = BeautifulSoup(xml_text, 'lxml')

list_of_lists = []   # Creating an empty list, which we will use to hold our rows (each row represented as a list)


try:
    collection_title = clean_text(soup.find('titleproper').get_text())
except:
    collection_title = xml_filename   # If the 'titleproper' field is missing for some reason,
                                      ## we'll use the XML filename instead.

for element in soup.find_all('did'):

    # unittitle
    try:
        unittitle = clean_text(element.find('unittitle').get_text())
    except:
        unittitle = ''
    
    # unitdate
    try:
        unitdate = clean_text(element.find('unitdate').get_text())
    except:
        unitdate = ''
    
    # container type and number
    try:
        container_type = element.find('container')['type']
    except:
        container_type = ''

    try:
        container_number = element.find('container').get_text()
    except:
        container_number = ''

    # extent
    try:
        extent = element.find('extent').get_text()
    except:
        extent = ''

    row = [collection_title, unittitle, unitdate, container_type, container_number, extent]

    list_of_lists.append(row)    ## Adding the row list we defined in the previous line to 'list_of_lists' 


list_of_lists[:15]   ## Outputting the first 15 rows in our list of lists

In [ ]:
## Almost there! Next we'll run the script above on each XML file in our list, creating a 
## master list of lists that we'll write to disk as a CSV in the next cell.

## Let's begin by re-loading our list of XML filenames:

os.chdir('/sharedfolder/LOC_Metadata')
xml_filenames = [item for item in os.listdir('./') if item[-2:]=='.2']  # Creating a list of XML filenames

list_of_lists = []  # Creating an empty list

## Now we'll extract metadata from the full batch of XML files. This may take a few moments to complete.

for xml_filename in xml_filenames:
    
    xml_text = open(xml_filename).read()
    
    soup = BeautifulSoup(xml_text, 'lxml')
    
    try:
        collection_title = clean_text(soup.find('titleproper').get_text())
    except:
        collection_title = xml_filename   # If the 'titleproper' field is missing for some reason,
                                          ## we'll use the XML filename instead.
    
    for element in soup.find_all('did'):
    
        # unittitle
        try:
            unittitle = clean_text(element.find('unittitle').get_text())
        except:
            unittitle = ''
        
        # unitdate
        try:
            unitdate = clean_text(element.find('unitdate').get_text())
        except:
            unitdate = ''
        
        # container type and number
        try:
            container_type = element.find('container')['type']
        except:
            container_type = ''
    
        try:
            container_number = element.find('container').get_text()
        except:
            container_number = ''
    
        # extent
        try:
            extent = element.find('extent').get_text()
        except:
            extent = ''
    
        row = [collection_title, unittitle, unitdate, container_type, container_number, extent]
    
        list_of_lists.append(row)


print(len(list_of_lists))   ## Printing the number of rows in our table

In [ ]:
## Finally, we write the extracted metadata to disk as a CSV called 'LOC_RS_Reduced_Metadata.csv'

out_path = "./LOC_RS_Reduced_Metadata.csv"  # The './' part is optional; it just means we're writing to 
                                            # the current working directory.

# Defining a list of column headers, which we will write as the first row in our CSV
column_headers = ['Collection Title', 'Unit Title', 'Unit Date', 'Container Type', 'Container Number', 'Extent']

import csv                               # Importing Python's built-in CSV input/output package

with open(out_path, 'w') as fo:          # Creating a tempory file stream object called 'fo' (my abbreviation for 'file out')
    csv_writer = csv.writer(fo)          # Initializing our CSV writer
    csv_writer.writerow(column_headers)  # Writing one row (our column headers)
    csv_writer.writerows(list_of_lists)  # Writing a list of lists as a sequence of rows

In [ ]:
## Go to 'sharedfolder' on your desktop and use LibreOffice or Excel to open your new CSV.

## As you scroll through the CSV file, you will probably see more formatting oddities you can fix 
## by tweaking the code above.